<a href="https://colab.research.google.com/github/tmkhang1999/Applied-Data-Science-Capstone/blob/master/Interactive_Dashboard_with_Ploty_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [ ]:
!pip install jupyter-dash -q

In [ ]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                 ],
                                             value='ALL',
                                             placeholder="ZSelect the launch Site",
                                             searchable=True
                                             ),
                                
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]
                                                ),
                                
                                html.Br(),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, 
                     values='class',
                     names='Launch Site',
                     title='Total Success Launches for All Launch Site')
    else:
        filtered_df = spacex_df.groupby(['Launch Site','class']).size().reset_index(name="count")
        fig = px.pie(filtered_df[filtered_df['Launch Site'] == entered_site],
                     values = "count",
                     names = "class",
                     title=f"Total Success Launches for site {entered_site}")
        
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))

def get_scatter_chart(entered_site, payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,
                       x='Payload Mass (kg)',
                       y='class',
                       color='Booster Version Category',
                       title='Success count on Payload mass for all sites')
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site],
                       x='Payload Mass (kg)',
                       y='class',
                       color='Booster Version Category',
                       title=f"Success count on Payload mass for site {entered_site}")
        
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', host='localhost', port=8000)

<IPython.core.display.Javascript object>